In [1]:
!pip install torch torchvision transformers

In [2]:
#importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import copy

## Training the model with early stopping and calbacks

In [3]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [4]:
# Define transformations
transform = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [5]:
# Load dataset
train_data = datasets.ImageFolder(root='/kaggle/input/braintumor/Training', transform=transform['train'])
test_data = datasets.ImageFolder(root='/kaggle/input/braintumor/Testing', transform=transform['test'])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)


In [6]:
class StrongCNN(nn.Module):
    def __init__(self):
        super(StrongCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 4)  # Assuming 4 classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

model = StrongCNN().to(device)

In [7]:
# Initialize loss, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, verbose=True)

In [8]:
# Early stopping parameters
early_stopping_patience = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
epochs_no_improve = 0
n_epochs = 30


In [9]:
# Training and validation functions
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, early_stopping_patience):
    global best_model_wts, best_acc, epochs_no_improve
    model = model.to(device)

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        val_loss, val_acc = validate_model(model, test_loader, criterion)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping_patience:
                print(f'Early stopping at epoch {epoch+1}')
                model.load_state_dict(best_model_wts)
                return model

    model.load_state_dict(best_model_wts)
    return model

In [10]:

def validate_model(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    return val_loss, accuracy


In [11]:
# Train the model
model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, early_stopping_patience)

Epoch 1/30, Loss: 0.8491, Val Loss: 0.6760, Val Acc: 0.7162
Epoch 2/30, Loss: 0.5801, Val Loss: 0.5723, Val Acc: 0.7574
Epoch 3/30, Loss: 0.5000, Val Loss: 0.5524, Val Acc: 0.7666
Epoch 4/30, Loss: 0.4653, Val Loss: 0.4154, Val Acc: 0.8330
Epoch 5/30, Loss: 0.4213, Val Loss: 0.3927, Val Acc: 0.8444
Epoch 6/30, Loss: 0.3936, Val Loss: 0.4169, Val Acc: 0.8413
Epoch 7/30, Loss: 0.3564, Val Loss: 0.3390, Val Acc: 0.8642
Epoch 8/30, Loss: 0.3306, Val Loss: 0.3131, Val Acc: 0.8650
Epoch 9/30, Loss: 0.3114, Val Loss: 0.3062, Val Acc: 0.8673
Epoch 10/30, Loss: 0.2891, Val Loss: 0.3085, Val Acc: 0.8780
Epoch 11/30, Loss: 0.2668, Val Loss: 0.2281, Val Acc: 0.9146
Epoch 12/30, Loss: 0.2511, Val Loss: 0.2408, Val Acc: 0.8986
Epoch 13/30, Loss: 0.2371, Val Loss: 0.1777, Val Acc: 0.9336
Epoch 14/30, Loss: 0.2257, Val Loss: 0.1743, Val Acc: 0.9367
Epoch 15/30, Loss: 0.2086, Val Loss: 0.2332, Val Acc: 0.9191
Epoch 16/30, Loss: 0.1737, Val Loss: 0.1672, Val Acc: 0.9443
Epoch 17/30, Loss: 0.1788, Val Lo

# Transfer learning with Hugging face model

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
from transformers import ViTForImageClassification, ViTFeatureExtractor

2024-07-01 09:20:55.427886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 09:20:55.428014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 09:20:55.559637: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [14]:
# Load ViT model and feature extractor
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=4)  # Assuming 4 classes


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Use DataParallel for multi-GPU training
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.to(device)

In [16]:
# Define transformations
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
    ]),
}

In [17]:
# Load dataset
train_data = datasets.ImageFolder(root='/kaggle/input/braintumor/Training', transform=transform['train'])
test_data = datasets.ImageFolder(root='/kaggle/input/braintumor/Testing', transform=transform['test'])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)


In [18]:
# Initialize loss, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, verbose=True)


In [19]:
# Early stopping parameters
early_stopping_patience = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
epochs_no_improve = 0
n_epochs = 30

In [20]:
# Mixed precision training scaler
scaler = torch.cuda.amp.GradScaler()

In [21]:
# Training and validation functions
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, early_stopping_patience):
    global best_model_wts, best_acc, epochs_no_improve

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(inputs).logits
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        val_loss, val_acc = validate_model(model, test_loader, criterion)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping_patience:
                print(f'Early stopping at epoch {epoch+1}')
                model.load_state_dict(best_model_wts)
                return model

    model.load_state_dict(best_model_wts)
    return model


In [22]:

def validate_model(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs).logits
                loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    return val_loss, accuracy

In [23]:
# Train the model
model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, early_stopping_patience)


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/30, Loss: 0.3054, Val Loss: 0.1266, Val Acc: 0.9641
Epoch 2/30, Loss: 0.0873, Val Loss: 0.0775, Val Acc: 0.9832
Epoch 3/30, Loss: 0.0630, Val Loss: 0.0553, Val Acc: 0.9825
Epoch 4/30, Loss: 0.0291, Val Loss: 0.0305, Val Acc: 0.9893
Epoch 5/30, Loss: 0.0290, Val Loss: 0.0436, Val Acc: 0.9893
Epoch 6/30, Loss: 0.0367, Val Loss: 0.0440, Val Acc: 0.9817
Epoch 7/30, Loss: 0.0313, Val Loss: 0.0196, Val Acc: 0.9947
Epoch 8/30, Loss: 0.0196, Val Loss: 0.0362, Val Acc: 0.9924
Epoch 9/30, Loss: 0.0282, Val Loss: 0.0891, Val Acc: 0.9741
Epoch 10/30, Loss: 0.0159, Val Loss: 0.0310, Val Acc: 0.9878
Epoch 11/30, Loss: 0.0278, Val Loss: 0.0414, Val Acc: 0.9840
Epoch 12/30, Loss: 0.0194, Val Loss: 0.0542, Val Acc: 0.9893
Epoch 00013: reducing learning rate of group 0 to 2.0000e-05.
Epoch 13/30, Loss: 0.0125, Val Loss: 0.0462, Val Acc: 0.9886
Epoch 14/30, Loss: 0.0050, Val Loss: 0.0383, Val Acc: 0.9924
Epoch 15/30, Loss: 0.0024, Val Loss: 0.0351, Val Acc: 0.9924
Epoch 16/30, Loss: 0.0026, Val L